In [ ]:
from google.colab import files
files.upload()

Saving intents.json to intents (1).json


{'intents.json': b'{\n  "intents": [\n    {\n      "tag": "greeting",\n      "patterns": [\n        "Hi",\n\t  "Hi Bot",\n\t  "Hola",\n\t  "Hi, how are you",\n        "How are you",\n        "Is anyone there?",\n        "Hello",\n        "Hey",\n        "What\'s up",\n        "Hi there",\n        "Hey there",\n        "Good morning",\n        "Good afternoon",\n        "Good evening",\n        "Howdy",\n        "Hello there"\n      ],\n      "responses": [\n        "What can i help you with today?",\n        "How can I help you today?"\n      ],\n      "context_set": ""\n    },\n    {\n      "tag": "goodbye",\n      "patterns": [\n        "Bye",\n        "See you later",\n        "Goodbye",\n        "Farewell",\n        "Take care",\n        "Catch you later",\n        "Goodbye for now",\n        "Bye for now",\n        "Talk to you later",\n        "See you soon",\n        "Take it easy",\n        "So long",\n        "Adios",\n        "Good night",\n        "Later",\n        "Have a g

In [ ]:
import gensim.downloader as api
# Load pre-trained word2vec model
model = api.load("word2vec-google-news-300")

In [ ]:
import json
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk
from tensorflow import keras
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

# Load the intents data
with open("intents.json", "r") as f:
    data = json.load(f)

# Preprocessing
vocab = set()
classes = []
documents = []

for intent in data["intents"]:
    for pattern in intent["patterns"]:
        pattern = pattern.rstrip("?")
        tokens = word_tokenize(pattern)
        tokens = [token.lower() for token in tokens if token not in set(stopwords.words("english"))]
        vocab.update(tokens)
        documents.append((tokens, intent["tag"]))
    if intent["tag"] not in classes:
        classes.append(intent["tag"])

# Generate word embeddings for each token
X = []
y = []
for doc in documents:
    vecs = []
    P = 0
    for token in doc[0]:
        if token in model.key_to_index:
            vecs.append(model[token])
    if vecs:
        vecs = np.mean(vecs, axis=0)
        X.append(vecs)
        y.append(classes.index(doc[1]))

# Note: The code snippet references a model variable, which should be defined or loaded with appropriate models before this code snippet.
# Please make sure to define or load the model before using this code.

# The code above generates X and y based on the intents data, representing the word embeddings and labels respectively.

# Please note that the vectorization and training steps are not included in this code snippet.
# Additional code is required to vectorize the sentences and train a model for intent classification.
# Once the model is trained, it can be used for predicting intents based on user input.
# The prediction code will depend on the specific model and vectorization technique used.

# It's important to note that the code snippet provided requires additional implementation steps before it can be run successfully.
# Please ensure that the necessary model, data, and vectorization steps are implemented appropriately.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
X = np.array(X)
y = np.array(y)


In [ ]:
X.shape

(287, 300)

In [ ]:

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)


In [ ]:
X_train.shape

(229, 300)

In [ ]:
len(X_train[1])

300

In [ ]:
from xgboost import XGBClassifier

# Create the model
model1 = XGBClassifier()

# Train the model
model1.fit(X_train, y_train)

# Evaluate the model
accuracy = model1.score(X_test, y_test)
print(f"Accuracy: {accuracy}")


Accuracy: 0.7586206896551724


In [ ]:
def vectorize_sentence(sentence, model):
    sentence = sentence.rstrip("?")
    # tokenize the sentence
    sentence_words = nltk.word_tokenize(sentence)
    # lemmatize each word and remove stopwords
    sentence_words = [lemmatizer.lemmatize(word.lower(), pos='v') for word in sentence_words if word not in set(stopwords.words("english"))]
    # create a vector representation
    vecs = []
    for word in sentence_words:
        if word in model.key_to_index:
            vecs.append(model[word])
    if vecs:
        vecs = np.mean(vecs, axis=0)
    else:
        vecs = np.zeros(model.vector_size)
    return vecs

print("Hi there, this is Genie. I'm so happy to chat with you today! 😊")
print("Could you please be more specific with your questions.")
print("I'm always trying to improve my communication skills and learn from your feedback. (type quit to end the conversation):")

while True:
    inp = input("You: ")  # Get user input
    if inp.lower() == "quit":
        for intent in data["intents"]:
            if intent["tag"] == "goodbye":
                print("Bot: " + random.choice(intent["responses"]))
        break  # Exit the loop and end the conversation

    vec = vectorize_sentence(inp, model)  # Vectorize the user input using the model's embeddings
    results = model1.predict(np.array([vec]))  # Make a prediction using the trained model
    if results[0] > 0.5:
        tag = classes[results[0]]
        for intent in data["intents"]:
            if intent["tag"] == tag:
                if intent["tag"] == "goodbye" and inp == 'quit':
                    print("Bot: " + random.choice(intent["responses"]))
                    print("Bot: Goodbye!")
                    break
                if intent["tag"] == "goodbye" and inp != 'quit':
                    print("Bot: I am evolving constantly. I apologize that I could not help you with your query. Please give us a call at  and our team will be happy to assist")
                else:
                    print("Bot: " + random.choice(intent["responses"]))
                    break
    else:
        print("Bot: I am evolving constantly. I apologize that I could not help you with your query. Please give us a call at  and our team will be happy to assist")

# The code above defines a function vectorize_sentence() to convert a sentence into a vector representation using the provided model's embeddings.
# It then initializes a conversation loop where user input is processed and classified using the trained model.
# The intent with the highest predicted probability is selected, and an appropriate response is printed based on the intent.

# Please note that the code snippet provided requires additional implementation steps before it can be run successfully.
# It assumes the availability of the model and associated embeddings, as well as appropriate data loading and preprocessing.
# Additionally, specific handling of the conversation loop and responses based on intents may require further customization to fit your use case.

Hi there, this is Genie. I'm so happy to chat with you today! 😊 
Could you please be more specific with your questions.
I'm always trying to improve my communication skills and learn from your feedback. (type quit to end the conversation):
You: How much time will it take for the order to be prepared?
Bot: Of course, You can place your order online at https://www.suggioota.com/order-online
You: I need to order for catering.
Bot: We will be more than happy to serve fresh food to make your event more memorable. Feel free to call us at (669)-224-0833
You: Is home Delivery available or pickup?
Bot: We have home delivery options through Doordash, Uber Eats, Grubhub, and LemonHat
You: what are the vegetarian options?
Bot: You can check out the food menu at https://www.suggioota.com/order-online
You: What drinks do you offer?
Bot: We are one of the Cloud Kitchens (K-31) located at 949 Ruff Dr, San Jose, CA 95110. Please follow the link to visit us: https://www.google.com/maps?q=949+Ruff+Dr,+Sa

In [ ]:
path='/content/drive/My Drive/Colab Notebooks' #Declare the path where the code and model file has been placed

In [ ]:
import pickle

# Save the model to disk
with open("model_XGBC_Word2vec.pickle", "wb") as f:
    pickle.dump(model1, f)